In [ ]:
import pandas as pd
import numpy as np
#import bootstrapped.bootstrap as bts
#import bootstrapped.stats_functions as bs_stats
import os
import scipy.stats as stats
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib
import seaborn as sns
import pylab as pl
from os import path
import itertools 
from itertools import combinations 

# settings for making nice pdfs
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.sans-serif'] = "Arial"
plt.rcParams['font.family'] = "sans-serif"

In [ ]:
#define helper functions
def round_sf(x,digits):
    if x != 0:
        x = float(np.format_float_positional(x, precision=digits, unique=False, fractional=False,trim='k'))
    return x

In [ ]:
base_folder_string = 'output_0310_tiling600_FullParamter_DenseDSBs_run'
folders = [foldername for foldername in os.listdir('Data/') if base_folder_string in foldername]


base_stochasticity = 1 # use to convert step to time

for m in range(len(folders)):
    data_dir = folders[m]
    print(data_dir)
    stats_file = [filename for filename in os.listdir('Data/'+data_dir) if filename.startswith("stats_df")]

    sep_proc_list = list(set([stats_file[i].split('stats_df_')[1].split('_superportion')[0] for i in range(len(stats_file))]))

    superportion_list = list(set([stats_file[i].split('superportion')[1].split('_superproc')[0] for i in range(len(stats_file))]))
    superportion_numerical_list = [float(a) for a in superportion_list]
    superportion_list_sorted = [x for _,x in sorted(zip(superportion_numerical_list,superportion_list))]
    superportion_numerical_sorted = [float(a) for a in superportion_list_sorted]
    superproc_list = list(set([stats_file[i].split('superproc')[1].split('_ctcf')[0] for i in range(len(stats_file))]))
    superproc_numerical_list = [float(a) for a in superproc_list]
    superproc_list_sorted = [x for _,x in sorted(zip(superproc_numerical_list,superproc_list))]
    superproc_numerical_sorted = [float(a) for a in superproc_list_sorted]
    ctcf_list = list(set([stats_file[i].split('ctcf')[1].split('_dsb')[0] for i in range(len(stats_file))]))
    ctcf_numerical_list = [float(a) for a in ctcf_list]
    ctcf_list_sorted = [x for _,x in sorted(zip(ctcf_numerical_list,ctcf_list))]
    ctcf_numerical_sorted = [float(a) for a in ctcf_list_sorted]
    dsb_list = list(set([stats_file[i].split('dsb')[1].split('_superloading')[0] for i in range(len(stats_file))]))
    dsb_numerical_list = [float(a) for a in dsb_list]
    dsb_list_sorted = [x for _,x in sorted(zip(dsb_numerical_list,dsb_list))]
    dsb_numerical_sorted = [float(a) for a in dsb_list_sorted ]
    superloading_list = list(set([stats_file[i].split('superloading')[1].split('_bs')[0] for i in range(len(stats_file))]))
    superloading_numerical_list = [float(a) for a in superloading_list]
    superloading_list_sorted = [x for _,x in sorted(zip(superloading_numerical_list,superloading_list))]
    superloading_numerical_sorted = [float(a) for a in superloading_list_sorted]
    bs_list = list(set([stats_file[i].split('bs')[1].split('.csv')[0] for i in range(len(stats_file))]))
    bs_numerical_list = [float(a) for a in bs_list]
    bs_list_sorted = [x for _,x in sorted(zip(bs_numerical_list,bs_list))]
    bs_numerical_sorted = [float(a) for a in bs_list_sorted]

    if m == 0:
        ctcf_bs_fp = np.zeros((len(folders),len(sep_proc_list),len(superportion_list),len(ctcf_list),len(dsb_list),len(superloading_list)))
        ctcf_bs_success =  np.zeros((len(folders),len(sep_proc_list),len(superportion_list),len(ctcf_list),len(dsb_list),len(superloading_list)))
        ctcf_bs_time0 =   np.zeros((len(folders),len(sep_proc_list),len(superportion_list),len(ctcf_list),len(dsb_list),len(superloading_list)))
        load_constrain_ratio = np.zeros((len(folders),len(sep_proc_list),len(superportion_list),len(ctcf_list),len(dsb_list),len(superloading_list)))
        extrusion_constrain_ratio = np.zeros((len(folders),len(sep_proc_list),len(superportion_list),len(ctcf_list),len(dsb_list),len(superloading_list)))
    
    count = 0 
    for i in range(len(sep_proc_list)):
        for j in range(len(superportion_list)):
            for k in range(len(ctcf_list)):
                for p in range(len(dsb_list)):
                    for q in range(len(superloading_list)):
                        proc = float(sep_proc_list[i].split('_proc')[1])
                        sep = float(sep_proc_list[i].split('sep')[1].split('_proc')[0])
                        if proc == 62.5:
                            par_combo = sep_proc_list[i] + '_superportion' + superportion_list_sorted[j] + '_superproc' + str(proc*20) + '_ctcf' + ctcf_list_sorted[k] + '_dsb' + dsb_list_sorted[p]+ '_superloading' + superloading_list_sorted[q]+ '_bs' + bs_list_sorted[0]
                        else:
                            par_combo = sep_proc_list[i] + '_superportion' + superportion_list_sorted[j] + '_superproc' + str(int(proc*20)) + '_ctcf' + ctcf_list_sorted[k] + '_dsb' + dsb_list_sorted[p]+ '_superloading' + superloading_list_sorted[q]+ '_bs' + bs_list_sorted[0]

                        filename = 'Data/' + data_dir + '/stats_df_' + par_combo + '.csv'

                        if path.exists(filename):
                            count+=1

                            stats_df = pd.read_csv(filename)
                            stats_df = stats_df.drop(columns='Unnamed: 0')
                            stats_df_threshold1 = stats_df.loc[stats_df['threshold']==1]
                            sep_proc_1st_pass = stats_df_threshold1['first_passage_time'].to_numpy()
                            if len(sep_proc_1st_pass)>0:
                                ctcf_bs_fp[m,i,j,k,p,q]=np.mean(sep_proc_1st_pass)*base_stochasticity/60 *2
                            else:
                                ctcf_bs_fp[m,i,j,k,p,q]=np.nan


                            portion_df = pd.read_csv('Data/'+ data_dir + '/restrained_df_' + par_combo + '.csv')
                            portion_df = portion_df.drop(columns='Unnamed: 0')
                            portion_df_threshold1 = portion_df.loc[portion_df['threshold']==1]
                            ctcf_bs_time0[m,i,j,k,p,q] = portion_df_threshold1['restrained proportion time0'].to_numpy()[0] * 100
                            ctcf_bs_success[m,i,j,k,p,q] = portion_df_threshold1['repaired proportion'].to_numpy()[0] * 100
                            
                            
                            ctcf_bs_capture[m,i,j,k,p,q] = ctcf_bs_success[m,i,j,k,p,q]/ctcf_bs_time0[m,i,j,k,p,q]
                            
                            D = 450 # average TAD size
                            d = sep
                            b = bs_numerical_sorted[0]
                            a0 = superportion_numerical_sorted[j]
                            s  = 20 #super vs normal proc ratio
                            w = ctcf_numerical_sorted[k]
                            r = dsb_numerical_sorted[p]
                            F = superloading_numerical_sorted[q]
                            v = 0.5
                            
                            constrain = proc/2/v #v = 0.5kb/second
                            boundfraction = d/2/D+1/2+d/b/w/(a0*s*proc+(1-a0)*proc)-np.sqrt((d/2/D+1/2+d/b/w/(a0*s*proc+(1-a0)*proc))**2-d/D)
                            alpha = a0*s/((1-a0)+a0*s)
                            beta = 1-(1-boundfraction)**2
                            proc_combined = beta*w*(a0*s*proc+(1-a0)*proc)+(1-beta)*(a0*s*proc+(1-a0)*proc)
                            l = 0.5*(10**(-0.08238 + 0.7258*np.log10(proc_combined /sep) - 
                             0.2514*(np.log10(proc_combined /sep))**2 - 
                             0.003995*(np.log10(proc_combined /sep))**3 + 
                             0.03445*(np.log10(proc_combined /sep))**4 - 
                             0.01077*(np.log10(proc_combined /sep))**5 + 
                             0.001371*(np.log10(proc_combined /sep))**6 - 
                             6.472*10**(-5)*(np.log10(proc_combined /sep))**7)*sep)

                            U = 10000 #dsb every 10kb
                            load = proc*(2*F+U-2)*d/2/v/(F+l/2-1)/U
                            constrain = proc/2/v
                            extrusion = l/4/v
                            load_constrain_ratio[m,i,j,k,p,q]= np.sqrt((alpha*load/(w*s*constrain)+(1-alpha)*load/(w*constrain))*(alpha*load/(w*s*r/(w*s+r)*constrain)+(1-alpha)*load/(w*r/(w+r)*constrain)))
                            extrusion_constrain_ratio[m,i,j,k,p,q]= np.sqrt((alpha*extrusion/(w*s*constrain)+(1-alpha)*extrusion/(w*constrain))*(alpha*extrusion/(w*s*r/(w*s+r)*constrain)+(1-alpha)*extrusion/(w*r/(w+r)*constrain)))
matplotlib.rcParams.update({'font.size': 22})       

fig, axs = plt.subplots(figsize=(20, 13))
sc = plt.scatter(np.mean(extrusion_constrain_ratio,axis=0),np.mean(load_constrain_ratio,axis=0),c=np.mean(ctcf_bs_success,axis=0),cmap='viridis',s=75,edgecolors='k', vmin=0,vmax=100)
plt.ylabel('Effective Extrusion Time / Effective Constrained Time')
plt.xlabel('Effective Loading Time / Effective Constrained Time')
cbar=plt.colorbar(sc)
cbar.ax.get_yaxis().labelpad = 20
cbar.ax.set_ylabel('Synpasis efficiency (%)', rotation=270)
plt.tick_params(direction='out', length=12, width=2)
plt.tick_params(axis='both', which='minor',length=6, width=2)
plt.yscale('log')
plt.xscale('log')

plt.savefig('Figures/'+'FullParameterScatter.pdf',format='pdf',bbox_inches='tight')
plt.show()


In [ ]:
# find the indexes for parameter combinations that achieve >=95% synapsis efficiency
perfect_index = np.argwhere(np.rint(np.mean(ctcf_bs_success,axis=0))>=95)

perfect_index 


In [ ]:
# mean synapsis time for the models achieving >=95% synapsis efficiency
print(np.mean(ctcf_bs_fp,axis=0)[5, 3, 3, 1, 3])
print(np.mean(ctcf_bs_fp,axis=0)[5, 3, 3, 2, 2])
print(np.mean(ctcf_bs_fp,axis=0)[5, 3, 3, 3, 2])

In [ ]:
# KO prediction
FoldReduction=np.zeros((len(perfect_index),perfect_index.shape[1]))
OriginalEfficiency=np.zeros((len(perfect_index),perfect_index.shape[1]))
NewEfficiency=np.zeros((len(perfect_index),perfect_index.shape[1]))
for i in range(perfect_index.shape[1]):
    for j in range(len(perfect_index)):
        index = perfect_index[j]
        newindex = np.copy(perfect_index[j])
        newindex[i]=0
        OriginalEfficiency[j,i]=np.mean(ctcf_bs_success,axis=0)[index[0],index[1],index[2],index[3],index[4]]
        NewEfficiency[j,i] =  np.mean(ctcf_bs_success,axis=0)[newindex[0],newindex[1],newindex[2],newindex[3],newindex[4]]
        FoldReduction[j,i]= 100*(np.mean(ctcf_bs_success,axis=0)[index[0],index[1],index[2],index[3],index[4]]-\
                             np.mean(ctcf_bs_success,axis=0)[newindex[0],newindex[1],newindex[2],newindex[3],newindex[4]])/np.mean(ctcf_bs_success,axis=0)[index[0],index[1],index[2],index[3],index[4]]

matplotlib.rcParams.update({'font.size': 22})
fig, axs = plt.subplots(1,1,figsize=(55, 15))

# width of the bars
barWidth = 0.2
r1 = np.arange(5)
r2 = [x + barWidth for x in r1]

bars1 = np.mean(FoldReduction,axis=0)
yer1 = stats.sem(FoldReduction,axis=0,nan_policy='omit')
bars2 = np.mean(NewEfficiency,axis=0)
yer2 = stats.sem(NewEfficiency,axis=0,nan_policy='omit')
bars3 = np.mean(OriginalEfficiency,axis=0)
yer3 = stats.sem(OriginalEfficiency,axis=0,nan_policy='omit')
bars1_sorted = [x for _,x in sorted(zip(bars1,bars1),reverse=True)]
yer1_sorted = [x for _,x in sorted(zip(bars1,yer1),reverse=True)]
bars2_sorted = [x for _,x in sorted(zip(bars1,bars2),reverse=True)]
yer2_sorted = [x for _,x in sorted(zip(bars1,yer2),reverse=True)]
bars3_sorted = [x for _,x in sorted(zip(bars1,bars3),reverse=True)]
yer3_sorted = [x for _,x in sorted(zip(bars1,yer3),reverse=True)]

label  =[ 'Lowering processivity/separation to 0.5', 'Knockout long-lived LEFs', 'Knockout BE stabilization' , 'Knockout DSB stabilization', 'Knockout targeted loading']
label_sorted = [x for _,x in sorted(zip(bars1,label),reverse=True)]

# Create blue bars
axs.bar(r1, bars3_sorted, width = barWidth, color = 'cyan', edgecolor = 'black', yerr=yer3_sorted, capsize=7, label=sep_proc_list[0],error_kw={'elinewidth':4,'capsize':10,'capthick':4})
axs.bar(r2, bars2_sorted, width = barWidth, color = 'blue', edgecolor = 'black', yerr=yer2_sorted, capsize=7, label=sep_proc_list[0],error_kw={'elinewidth':4,'capsize':10,'capthick':4})


# general layout
axs.set_xticks([r + barWidth*1.5 for r in range(len(bars1))])
axs.set_xticklabels(label_sorted)
axs.set_ylabel('Synapsis efficiency(%)')
plt.savefig('Figures/'+'KnockOut.pdf',format='pdf',bbox_inches='tight')

In [ ]:
# efficiency after KO
np.round(bars2_sorted)

In [ ]:
# KO prediction for synapsis time
FoldReduction=np.zeros((len(perfect_index),perfect_index.shape[1]))
OriginalSynapsisTime=np.zeros((len(perfect_index),perfect_index.shape[1]))
NewSynapsisTime=np.zeros((len(perfect_index),perfect_index.shape[1]))
for i in range(perfect_index.shape[1]):
    for j in range(len(perfect_index)):
        index = perfect_index[j]
        newindex = np.copy(perfect_index[j])
        newindex[i]=0
        OriginalSynapsisTime[j,i]=np.mean(ctcf_bs_fp,axis=0)[index[0],index[1],index[2],index[3],index[4]]
        NewSynapsisTime[j,i] =  np.mean(ctcf_bs_fp,axis=0)[newindex[0],newindex[1],newindex[2],newindex[3],newindex[4]]
        FoldReduction[j,i]= 100*(np.mean(ctcf_bs_fp,axis=0)[index[0],index[1],index[2],index[3],index[4]]-\
                             np.mean(ctcf_bs_fp,axis=0)[newindex[0],newindex[1],newindex[2],newindex[3],newindex[4]])/np.mean(ctcf_bs_fp,axis=0)[index[0],index[1],index[2],index[3],index[4]]

matplotlib.rcParams.update({'font.size': 22})
fig, axs = plt.subplots(1,1,figsize=(55, 15))

# width of the bars
barWidth = 0.2
r1 = np.arange(5)
r2 = [x + barWidth for x in r1]

bars1 = np.mean(FoldReduction,axis=0)
yer1 = stats.sem(FoldReduction,axis=0,nan_policy='omit')
bars2 = np.mean(NewSynapsisTime,axis=0)
yer2 = stats.sem(NewSynapsisTime,axis=0,nan_policy='omit')
bars3 = np.mean(OriginalSynapsisTime,axis=0)
yer3 = stats.sem(OriginalSynapsisTime,axis=0,nan_policy='omit')
bars1_sorted = [x for _,x in sorted(zip(bars1,bars1),reverse=True)]
yer1_sorted = [x for _,x in sorted(zip(bars1,yer1),reverse=True)]
bars2_sorted = [x for _,x in sorted(zip(bars1,bars2),reverse=True)]
yer2_sorted = [x for _,x in sorted(zip(bars1,yer2),reverse=True)]
bars3_sorted = [x for _,x in sorted(zip(bars1,bars3),reverse=True)]
yer3_sorted = [x for _,x in sorted(zip(bars1,yer3),reverse=True)]

label  =[ 'Lowering processivity/separation to 0.5', 'Knockout long-lived LEFs', 'Knockout BE stabilization' , 'Knockout DSB stabilization', 'Knockout targeted loading']
label_sorted = [x for _,x in sorted(zip(bars1,label),reverse=True)]

# Create blue bars
axs.bar(r1, bars3_sorted, width = barWidth, color = 'cyan', edgecolor = 'black', yerr=yer3_sorted, capsize=7, label=sep_proc_list[0],error_kw={'elinewidth':4,'capsize':10,'capthick':4})
axs.bar(r2, bars2_sorted, width = barWidth, color = 'blue', edgecolor = 'black', yerr=yer2_sorted, capsize=7, label=sep_proc_list[0],error_kw={'elinewidth':4,'capsize':10,'capthick':4})


# general layout
axs.set_xticks([r + barWidth*1.5 for r in range(len(bars1))])
axs.set_xticklabels(label_sorted)
axs.set_ylabel('Synapsis Time(min)')
plt.savefig('Figures/'+'KnockOut.pdf',format='pdf',bbox_inches='tight')

In [ ]:
# synapsis time before KO
bars3_sorted

In [ ]:
# synapsis time after KO each mechanism
bars2_sorted

In [ ]:
# %increase in synapsis time upon lowering proc/sep ratio from 2 to 0.5
(21.267600612315274-12.721139507149482)/ 12.721139507149482*100

In [ ]:
# %increase in synapsis time upon knocking out DSB stabilization
(17.74008007920686-12.721139507149482)/ 12.721139507149482*100

In [ ]:
# %increase in synapsis time upon knocking out targeted loading
(35.74008007920686-12.721139507149482)/ 12.721139507149482*100

In [ ]:
#double knockout index
double = [(0,1),(0,2),(0,3),(0,4),(1,2),(1,3),(1,4),(2,3),(2,4),(3,4)]
FoldReduction=np.zeros((len(perfect_index),10))
OriginalEfficiency=np.zeros((len(perfect_index),10))
NewEfficiency=np.zeros((len(perfect_index),10))
for i,pair in enumerate(double):
    for j in range(len(perfect_index)):
        index = perfect_index[j]
        newindex = np.copy(perfect_index[j])
        newindex[pair[0]]=0
        newindex[pair[1]]=0
        OriginalEfficiency[j,i]=np.mean(ctcf_bs_success,axis=0)[index[0],index[1],index[2],index[3],index[4]]
        NewEfficiency[j,i] =  np.mean(ctcf_bs_success,axis=0)[newindex[0],newindex[1],newindex[2],newindex[3],newindex[4]]
        FoldReduction[j,i]= 100*(np.mean(ctcf_bs_success,axis=0)[index[0],index[1],index[2],index[3],index[4]]-\
                             np.mean(ctcf_bs_success,axis=0)[newindex[0],newindex[1],newindex[2],newindex[3],newindex[4]])/np.mean(ctcf_bs_success,axis=0)[index[0],index[1],index[2],index[3],index[4]]

matplotlib.rcParams.update({'font.size': 22})
fig, axs = plt.subplots(1,1,figsize=(55, 10))

# width of the bars
barWidth = 0.2
r1 = np.arange(10)
r2 = [x + barWidth for x in r1]

bars1 = np.mean(FoldReduction,axis=0)
yer1 = stats.sem(FoldReduction,axis=0,nan_policy='omit')
bars2 = np.mean(NewEfficiency,axis=0)
yer2 = stats.sem(NewEfficiency,axis=0,nan_policy='omit')
bars3 = np.mean(OriginalEfficiency,axis=0)
yer3 = stats.sem(OriginalEfficiency,axis=0,nan_policy='omit')
bars1_sorted = [x for _,x in sorted(zip(bars1,bars1),reverse=True)]
yer1_sorted = [x for _,x in sorted(zip(bars1,yer1),reverse=True)]
bars2_sorted = [x for _,x in sorted(zip(bars1,bars2),reverse=True)]
yer2_sorted = [x for _,x in sorted(zip(bars1,yer2),reverse=True)]
bars3_sorted = [x for _,x in sorted(zip(bars1,bars3),reverse=True)]
yer3_sorted = [x for _,x in sorted(zip(bars1,yer3),reverse=True)]

label  =[ 'Lowering processivity/separation to 0.5+Knockout long-lived LEFs', 'Lowering processivity/separation to 0.5+Knockout BE stabilization','Lowering processivity/separation to 0.5+Knockout DSB stabilization', 'Lowering processivity/separation to 0.5+Knockout targeted loading','Knockout long-lived LEFs+Knockout BE stabilization', 'Knockout long-lived LEFs+Knockout DSB stabilization','Knockout long-lived LEFs+Knockout targeted loading','Knockout BE stabilization + Knockout DSB stabilization' ,'Knockout BE stabilization + Knockout targeted loading' , 'Knockout DSB stabilization + Knockout targeted loading']
label_sorted = [x for _,x in sorted(zip(bars1,label),reverse=True)]

# Create blue bars
axs.bar(r1, bars3_sorted, width = barWidth, color = 'cyan', edgecolor = 'black', yerr=yer3_sorted, capsize=7, label=sep_proc_list[0],error_kw={'elinewidth':4,'capsize':10,'capthick':4})
axs.bar(r2, bars2_sorted, width = barWidth, color = 'blue', edgecolor = 'black', yerr=yer2_sorted, capsize=7, label=sep_proc_list[0],error_kw={'elinewidth':4,'capsize':10,'capthick':4})


# general layout
axs.set_xticks([r + barWidth*1.5 for r in range(len(bars1))])
axs.set_xticklabels(label_sorted, rotation=30)
axs.set_ylabel('% Reduction in synapsis efficiency')
plt.savefig('Figures/'+'DoubleKnockOut.pdf',format='pdf',bbox_inches='tight')